In [1]:
!python -V

Python 3.9.19


In [6]:
import pandas as pd

In [7]:
import pickle

In [8]:
import seaborn as sns
import matplotlib.pyplot as plt

In [80]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from datetime import timedelta


In [204]:

def find_header_row(file_path):
    # Load the Excel file
    df_raw = pd.read_excel(file_path)
    
    # Iterate through rows to find the header row
    for i, row in df_raw.iterrows():
        if "BOROUGH" in row.values:  # Replace with any known column name
            return i
    
    return None

def excelToDf(file_path):
    start_row_index = find_header_row(file_path)
    df = pd.read_excel(file_path,skiprows=start_row_index+1)
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    df.rename(columns={'building_class_at_time_of_sale': 'building_class'}, inplace=True)
    df['zip_code'] = df['zip_code'].astype(str)
    df= df[(df.sale_price> 1200000) &(df.sale_price< 5000000)&(df.land_square_feet>0) &(df.gross_square_feet>0) & 
           (df.total_units>0) &(len(df.zip_code)>0) & (df.year_built>1950)]
    df['total_square_footage'] = df.land_square_feet + df.gross_square_feet


    # categorical = ['zip_code']
    # df[categorical] = df[categorical].astype(str)
    return df

df = excelToDf('./data/rollingsales_brooklyn_june_2024.xlsx')


In [205]:
df.dtypes

borough                               int64
neighborhood                         object
building_class_category              object
tax_class_at_present                 object
block                                 int64
lot                                   int64
easement                            float64
building_class_at_present            object
address                              object
apartment_number                     object
zip_code                             object
residential_units                   float64
commercial_units                    float64
total_units                         float64
land_square_feet                    float64
gross_square_feet                   float64
year_built                          float64
tax_class_at_time_of_sale             int64
building_class                       object
sale_price                            int64
sale_date                    datetime64[ns]
total_square_footage                float64
dtype: object

In [206]:
def split_data():
    df['sale_date'] = pd.to_datetime(df['sale_date'], errors='coerce')
    max_date = df['sale_date'].max()
    
    # Determine the start date for the two latest months
    two_months_ago = max_date - timedelta(days=60)
    train_data = df[df.sale_date<two_months_ago]
    val_data = df[df.sale_date>=two_months_ago]
    return (train_data,val_data)


train_data,val_data = split_data()


In [207]:
train_data.size


6666

In [208]:
val_data.size

1210

In [224]:
categorical = ["building_class_category","building_class"]
numerical = ["total_units"]


train_dicts = train_data[categorical + numerical].to_dict(orient='records')
train_dicts

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'sale_price'
y_train = train_data[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

val_dicts = val_data[categorical+numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_val = val_data[target].values
y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

/Users/konstantinos/.local/share/virtualenvs/notebook_experimenting--TWBekga/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


np.float64(935242.7504082854)

In [223]:
mean_absolute_percentage_error(y_val, y_pred)

np.float64(0.27195524138108734)

In [16]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)